In [1]:
import gym
import warnings
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Suppress DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow as tf

# List all GPUs available to TensorFlow
gpus = tf.config.list_physical_devices('GPU')

# Check if any GPUs are available
if len(gpus) > 0:
    print("GPU is available")
    for gpu in gpus:
        print(f"GPU device: {gpu}")
else:
    print("GPU is not available")

2023-10-18 17:42:39.620823: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-18 17:42:41.964994: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 17:42:48.732361: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available
GPU device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU')


In [4]:
# Question 1: Cartpole-v0
env = gym.make(
    "CartPole-v0",
    # render_mode="human"
    )

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)


def CartPole_RL(model, discount_factor=0.95, num_choices=2):
    
    # Save rewards for visualization later
    episode_rewards = []

    for episode in range(1000):
        # print(f"CartPole-v0, episode {episode}")
        # Initiate one episode
        observation, info = env.reset()

        obs_history = []
        reward_history = []
        action_history = []

        terminated = False
        truncated = False



        # Roll out one episode
        while (not terminated) and (not truncated):

            # Make a prediction based on the state
            probabilities = model.predict(np.array([observation]), verbose=0)[0]
            # print(probabilities)

            # This line stochastically samples an action based on the probabilities
            action = np.random.choice(num_choices, p=probabilities)
            # print(action)

            obs_history.append(observation)
            action_history.append(action)

            observation, reward, terminated, truncated, info = env.step(action)
            print(observation, reward, terminated, truncated, info)

            reward_history.append(reward)
            


        
        # Discount rewards
        discounted_rewards = []
        cumulative_reward = 0
        for reward in reversed(reward_history):
            cumulative_reward = reward + discount_factor * cumulative_reward
            discounted_rewards.insert(0, cumulative_reward)

        # print(f"Cumulative Reward History: {discounted_rewards}")


        # print(f"Frame: {count_frame}")
        # print(f"Obs History: {obs_history}")
        # print(f"Reward History: {reward_history}")
        # print(f"Action History: {action_history}")



        def adjust_weights(obs_history, action_history, discounted_rewards):
            
            with tf.GradientTape() as tape:
                # Predict action probabilities
                action_probabilities = model(np.array(obs_history))
                # print(action_probabilities)

                # Get probabilities of actions that were taken
                indices = list(zip(range(len(action_history)), action_history))
                # print(indices)
                chosen_action_probs = tf.gather_nd(action_probabilities, indices)
                # print(chosen_action_probs)

                # Compute loss
                loss = -tf.math.log(chosen_action_probs) * discounted_rewards
                loss = tf.reduce_mean(loss)

                
            # Calculate gradients
            grads = tape.gradient(loss, model.trainable_variables)
            # print(grads)
            
            # Apply gradients to model weights
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            # # Print updated model weights
            # for var in model.trainable_variables:
            #     print(var.name, var.numpy()[1])


        adjust_weights(obs_history=obs_history, action_history=action_history, discounted_rewards=discounted_rewards)


        # Append episode rewards for plotting
        total_reward = sum(reward_history)
        episode_rewards.append(total_reward)
        print(f"CartPole-v0 episode {episode}, reward sum: {total_reward}")

        # clear_output(wait=True)
        
    env.close()

    # After the for loop
    plt.plot(episode_rewards)
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.show()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation="relu", input_shape=(4,)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(2, activation="softmax")
])

CartPole_RL(model)

### Question 2 (Pong)

In [4]:
import gym
import os
import warnings
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Suppress DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def check_gpu_availability():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print("GPU is available")
        for gpu in gpus:
            print(f"GPU device: {gpu}")
    else:
        print("GPU is not available")

check_gpu_availability()

GPU is available
GPU device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU')


In [6]:
def preprocess(image, downsample_factor=2):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 2D float array """
    image = image[35:195] # crop
    image = image[::downsample_factor,::downsample_factor,0] # downsample by factor of 2
    image[image == 144] = 0 # erase background (background type 1)
    image[image == 109] = 0 # erase background (background type 2)
    image[image != 0] = 1 # everything else (paddles, ball) just set to 1
    # In preprocess function
    return np.reshape(image.astype(np.float64).ravel(), [80, 80])


def build_model(input_shape=(80, 80, 1), num_choices=2, reg=0.0001):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(reg), input_shape=input_shape),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(reg)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_choices, activation="softmax")
    ])
    return model

def select_action(model, observation):
    probabilities = model.predict(np.array([observation]), verbose=0)[0]
    action = np.random.choice([2, 3], p=probabilities)  # 2 is RIGHT and 3 is LEFT
    return action

def compute_discounted_rewards(reward_history, discount_factor=0.99):
    discounted_rewards, cumulative_reward = [], 0
    for reward in reversed(reward_history):
        cumulative_reward = reward + discount_factor * cumulative_reward
        discounted_rewards.insert(0, cumulative_reward)
    return discounted_rewards

def adjust_weights(model, optimizer, obs_history, action_history, discounted_rewards):
    with tf.GradientTape() as tape:
        probs=model(np.array(obs_history))
        indices=tf.stack([tf.range(len(action_history),dtype=tf.int32),tf.convert_to_tensor(action_history,dtype=tf.int32)],axis=1)
        chosen_probs=tf.gather_nd(probs,indices)
        loss=-tf.math.log(chosen_probs)*discounted_rewards
        loss=tf.reduce_mean(loss)
    grads=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))

def Pong_RL():
    env = gym.make("Pong-v0")
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model = build_model()
    episode_rewards = []
    episode = 0

    consecutive_21_rewards = 0  # Count number of 21 occurences
    should_train = True  # Initialize flag for training

    while True:
        observation, info = env.reset()
        obs_history, reward_history, action_history = [], [], []

        terminated = False
        truncated = False

        while not terminated and not truncated:
            processed_observation = preprocess(observation)
            action = select_action(model, processed_observation)
            obs_history.append(processed_observation)
            action_history.append(action)

            observation, reward, terminated, truncated, info = env.step(action)
            reward_history.append(reward)

        discounted_rewards = compute_discounted_rewards(reward_history)

        total_reward = sum(reward_history)
        episode_rewards.append(total_reward)

        moving_num, window = 0, 100
        if episode >= window-1:
            moving_avg = np.mean(episode_rewards[-window:])
            print(f"Pong-v0 episode {episode}, reward sum: {total_reward}, last {window} avg: {moving_avg:.2f}")
            
            if moving_avg > moving_num:
                print(f"Stopping as the last {window}-episode moving average is greater than {moving_num}")
                break
        else:
            print(f"Pong-v0 episode {episode}, reward sum: {total_reward}")

        ### TRAINING STOP

        if total_reward == 21:  # Check for consecutive rewards of 21
            consecutive_21_rewards += 1
            if consecutive_21_rewards >= 10 and should_train == True:
                print("Stopping training as the reward has been 21 for 10 episodes in a row")
                should_train = False  # Set flag to False

                # Check if folder exists, if not create it
                if not os.path.exists("saved_model"):
                    os.makedirs("saved_model")

                # Save the model
                model.save("saved_model/pong_model")
                
        else:
            consecutive_21_rewards = 0  # Reset the counter if the reward is not 21

        # Modify the weight adjustment to respect the should_train flag
        if should_train:
            adjust_weights(model, optimizer, obs_history, action_history, discounted_rewards)

        episode += 1

    env.close()

    # Save Results
    if not os.path.exists('artifacts'):
        os.makedirs('artifacts')

    # Calculate moving average of rewards
    window = 100  # Size of the window for calculating moving average
    moving_avgs = [np.mean(episode_rewards[max(0, i - window + 1):i+1]) for i in range(len(episode_rewards))]

    plt.plot(episode_rewards, label='Total Reward')
    plt.plot(moving_avgs, label=f'{window}-Episode Moving Average')
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.legend()
    plt.savefig('artifacts/pong_rewards.png')

if __name__ == "__main__":
    check_gpu_availability()
    Pong_RL()
    

GPU is available
GPU device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')
GPU device: PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU')


Error: We're Unable to find the game "Pong". Note: Gym no longer distributes ROMs. If you own a license to use the necessary ROMs for research purposes you can download them via `pip install gym[accept-rom-license]`. Otherwise, you should try importing "Pong" via the command `ale-import-roms`. If you believe this is a mistake perhaps your copy of "Pong" is unsupported. To check if this is the case try providing the environment variable `PYTHONWARNINGS=default::ImportWarning:ale_py.roms`. For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management

In [11]:
Pong_RL()

c:\Users\nuke2\Desktop\NW Work\Fall_02 Work\MSIA-FQ2\.venv\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(


ValueError: cannot reshape array of size 2916 into shape (80,80)